<a href="https://colab.research.google.com/github/chutimaq/bsc_dpdm22/blob/main/Final_Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## นำเข้าข้อมูล

In [ ]:
import pandas as pd
import os

from sklearn.model_selection import train_test_split

from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.neural_network import MLPClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score


from sklearn.tree import plot_tree 
from sklearn.metrics import classification_report

from sklearn.cluster import KMeans

import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.metrics import confusion_matrix
from sklearn.metrics import ConfusionMatrixDisplay

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
train = pd.read_csv('/content/drive/MyDrive/bsc_DPDM_data/Project/train.csv')

In [ ]:
test = pd.read_csv('/content/drive/MyDrive/bsc_DPDM_data/Project/test.csv')

In [ ]:
train.head(5)

In [ ]:
test.head(5)

#Data preparation

In [ ]:
train.describe()

In [ ]:
train[['m_dep','px_height','sc_w']].describe()

ไฟล์ train มี features m_dep, px height และ sc_w ที่มีค่า min ไม่สมเหตุสมผล 

min -> m_dep = 0.10,px height = 0 และ sc_w = 0

In [ ]:
test.describe()

ไฟล์ test มี features m_dep, px height และ sc_w ที่มีค่า min ไม่สมเหตุสมผล 

min -> m_dep = 0.10,px height = 0 และ sc_w = 0

In [ ]:
test[['m_dep','px_height','sc_w']].describe()

## Data Cleaning


*   ปรับค่า min ใหม่
*   Check Missing Values



### ปรับค่า min ใหม่

กำหนดขั้นต่ำของความลึก (m_dep) คือ 0.5 เซนติเมตร

In [ ]:
#train
below_threshold = train['m_dep'][train['m_dep'] < 0.5] #หา m_dep ที่น้อยกว่า 0.5
len(below_threshold)

In [ ]:
#Replace values below 0.5 cm with 0.5 cm
train['m_dep'][train['m_dep'] < 0.5] = 0.5

In [ ]:
#test
below_threshold1 = test['m_dep'][test['m_dep'] < 0.5] 
len(below_threshold1)

In [ ]:
# Replace values below 0.5 cm with 0.5 cm
test['m_dep'][test['m_dep'] < 0.5] = 0.5

กำหนดขั้นต่ำความละเอียดพิกเซล (px_height) คือ 65 พิกเซล



In [ ]:
#train
below_threshold2 = train['px_height'][train['px_height'] < 65] #หา px_height ที่น้อยกว่า 0.5
len(below_threshold2)

In [ ]:
# Replace values below 65 pixels with 65 pixels
train['px_height'][train['px_height'] < 65] = 65

In [ ]:
#test
below_threshold3 = test['px_height'][test['px_height'] < 65]
len(below_threshold3)

In [ ]:
# Replace values below 65 pixels with 65 pixels
test['px_height'][test['px_height'] < 65] = 65

กำหนดขั้นต่ำของความกว้างหน้าจอ (sc_w) คือ 1 นิ้ว = 2.54 เซนติเมตร

In [ ]:
#train
below_threshold4 = train['sc_w'][train['sc_w'] < 2.54]
len(below_threshold4)

In [ ]:
# Replace values below 2.54 cm with 2.54 cm
train['sc_w'][train['sc_w'] < 2.54] = 2.54

In [ ]:
#test
below_threshold5 = test['sc_w'][test['sc_w'] < 2.54]
len(below_threshold5)

In [ ]:
# Replace values below 2.54 cm with 2.54 cm
test['sc_w'][test['sc_w'] < 2.54] = 2.54

In [ ]:
train[['m_dep','px_height','sc_w']].describe()

In [ ]:
test[['m_dep','px_height','sc_w']].describe()

### check missing

In [ ]:
train.isnull().sum()

In [ ]:
test.isnull().sum()

ไม่พบ missing

## Correlation Analysis

In [ ]:
plt.figure(figsize=(20,15))
sns.heatmap(train.corr(), annot=True, cmap='Blues')
plt.title('Correlation Between The Features')
plt.show()

จาก heatmap จะพบว่า
*   หน่วยความจำ และ ช่วงราคา มีความสัมพันธ์กันอย่างมาก 
*   หน่วยความจำ, แบตเตอรี่, ความกว้างความละเอียดของภาพ และ ความสูงของความละเอียดของภาพ มีความสัมพันธ์กับช่วงราคา
*   4G และ 3G, กล้องหน้า และ กล้องหลัก, ความกว้างความละเอียดของภาพ และ ความสูงของความละเอียดของภาพ และ ความสูงหน้าจอมือถือ และ ความกว้างหน้าจอมือถือ มีความสัมพันธ์ในระดับปานกลาง



# **classification**
การคาดการณ์ช่วงราคาของโทรศัพท์มือถือโดยการสร้างแบบจำลองที่คำนึงถึงคุณลักษณะต่างๆ 
ที่มีอยู่ในชุดข้อมูล เราจะใช้วิธี Decision Tree, K-Nearest Neighbors
และ Naive Bayes 

In [ ]:
x = train.drop('price_range', axis=1) # drop column 'price_range' ให้เหลือแค่ Feature
y = train['price_range'].values.reshape(-1, 1) #Target : price_range

In [ ]:
# แบ่งข้อมูลออกเป็น train 80% test 20% 
X_train, X_test, y_train, y_test = train_test_split(x,y,test_size=0.2,random_state=2) # random_state ทำให้การสุ่มแต่ละครั้งมีผลการสุ่มเหมือนเดิม

## DecisionTree

In [ ]:
#  gridsearch หาพารามิเตอร์ที่ดีที่สุด
clf_dt = DecisionTreeClassifier()

max_depth = range(3, 21)
criterion = ['entropy', 'gini']
random_state = range(1, 8)

parameters = {'max_depth': max_depth, 'criterion': criterion, 'random_state': random_state}

gridsearch = GridSearchCV(estimator=clf_dt,
                          param_grid=parameters,
                          scoring='accuracy',
                          cv=10,
                          verbose=1,
                          n_jobs=-1)

gridsearch.fit(X_train, y_train)

print('Best parameters:', gridsearch.best_params_)

In [ ]:
# Define the decision tree model
clf = DecisionTreeClassifier(criterion = 'entropy', max_depth=7, random_state=2,)
# Fit the model to the training set
clf.fit(X_train, y_train)
#predict
y_pred = clf.predict(X_test)
# Calculate the accuracy of the model
acc_dt = accuracy_score(y_test, y_pred)
print(f'Decision Tree accuracy score: {acc_dt:.4f}');

In [ ]:
feature_names = x.columns
target_names = ['low cost', 'medium cost', 'high cost', 'very high cost']

fig = plt.figure(figsize = (30, 15))
Plot = plot_tree(clf, feature_names = feature_names, class_names = target_names, filled = True)

### Evaluation

In [ ]:
print(classification_report(y_test, y_pred))

###Confusion Matrix

In [ ]:
cm = confusion_matrix(y_test, y_pred)

disp = ConfusionMatrixDisplay(confusion_matrix=cm)
disp.plot(cmap=plt.cm.Blues)
plt.show()

## K-Nearest Neighbors

หา parameters ที่ดีที่สุด (K (1,3,5))

In [ ]:
# Define the hyperparameters to tune
param_grid = {'n_neighbors': [1, 3, 5]}

# Define the classifier
knn = KNeighborsClassifier()

# Perform grid search
grid_search = GridSearchCV(knn, param_grid=param_grid, cv=5, n_jobs=-1)
grid_search.fit(X_train, y_train)

# Print the best parameters
print('Best parameters:', grid_search.best_params_)

In [ ]:
# Instantiate the model
knn = KNeighborsClassifier(n_neighbors=5)

# Fit the model to the training set
knn.fit(X_train,y_train)

#predict
y_pred = knn.predict(X_test)

# Calculate the accuracy of the model
acc_knn = accuracy_score(y_test, y_pred)
print(f'KNN accuracy score: {acc_knn:.4f}');

### Evaluation

In [ ]:
print(classification_report(y_test, y_pred))

###Confusion Matrix

In [ ]:
cm = confusion_matrix(y_test, y_pred)

disp = ConfusionMatrixDisplay(confusion_matrix=cm)
disp.plot(cmap=plt.cm.Blues)
plt.show()

## Naive Bayes

In [ ]:
# Naive Bayes 
# Instantiate the model
nb = GaussianNB()

# Fit the model to the training set
nb.fit(X_train, y_train)

# Make predictions on the validation set
y_pred = nb.predict(X_test)

# Evaluate the performance of the model
acc_nb = accuracy_score(y_test, y_pred)
print(f'Naive Bayes accuracy score: {acc_nb:.4f}');

### Evaluation

In [ ]:
print(classification_report(y_test, y_pred))

###Confusion Matrix

In [ ]:
cm = confusion_matrix(y_test, y_pred)

disp = ConfusionMatrixDisplay(confusion_matrix=cm)
disp.plot(cmap=plt.cm.Blues)
plt.show()

เปรียบเทียบค่า accuracy score ของแต่ละ model

In [ ]:
print(f'Decision Tree accuracy score: {acc_dt:.4f}');
print(f'KNN accuracy score: {acc_knn:.4f}');
print(f'Naive Bayes accuracy score: {acc_nb:.4f}');

##Model Evaluation

จากการเปรียบเทียบค่า accuracy score Model KNN สามารถคาดการณ์ได้แม่นยำที่สุด เพราะมีค่า accuracy score สูงที่สุด

## Predicting Values for test.csv
จากที่ KNN ได้ accuracy score สูงที่สุด จึงนำ model KNN มาคาดการณ์ช่วงราคาสำหรับโทรศัพท์มือถือเครื่องใหม่ในไฟล์ test

In [ ]:
test_dt = test.drop('id',axis=1)
test_dt.shape

In [ ]:
test_pred = knn.predict(test_dt)

In [ ]:
test_dt['predicted_price'] = test_pred

In [ ]:
test_dt['predicted_price'].replace({0: 'low cost',1: 'medium cost', 2: 'high cost', 3:'very high cost'}, inplace=True)

In [ ]:
test_dt

# **K - means cluttering**
Clustering การจัดกลุ่มจะแตกต่างจากการแบ่งประเภทข้อมูล
(Classification) โดยจะแบ่งกลุ่มข้อมูลจากความคล้าย โดยไม่มีการกำหนดคลาสประเภทข้อมูลไว้ก่อนหรือไม่ทราบจำนวน
กลุ่มล่วงหน้า เป็นการเรียนรู้แบบไม่มีผู้สอน (unsupervised
classification) 

ต้องเป็นตัวแปรเชิงปริมาณ (Quantitative)
  - สเกลอันตรภาค (Interval Scale)
  - สเกลอัตราส่วน (Ratio Scale

In [ ]:
dt_test = pd.read_csv('/content/drive/MyDrive/bsc_DPDM_data/Project/test.csv')

In [ ]:
dt_train = pd.read_csv('/content/drive/MyDrive/bsc_DPDM_data/Project/train.csv')

In [ ]:
# drop price ออก
#kmeandata=df.drop('price_range',axis=1) ไม่ใช้
#kmeandata =train ไม่ใช้ 

kmeandata=pd.concat([dt_test,dt_train]) #เพื่อแบ่งเท่าๆกัน ใช้train set , test set 3000 rows
kmeandata =kmeandata.drop('price_range',axis=1) #drop price_range ที่ train,test มีไม่เหมือนกัน 

In [ ]:
kmeandata #data ที่ใช้ในการทำ k Mean clusttering

In [ ]:
kmeandata.columns

select columns to group >> 'battery_power', 'px_height'

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans

# Load data into a DataFrame
df = kmeandata

# Extract features to cluster on
X = df[['battery_power', 'px_height']].values 
#เพื่อจัดกลุ่ม ว่าจะตั้งราคาโดยจัดกลุ่มตาม ความจุแบตและ พิกเซล ของควาทมสูงจอ?

# Choose number of clusters
k = 6

# Run K-means clustering
kmeans = KMeans(n_clusters=k, random_state=42).fit(X)

# Get cluster assignments and centroids
labels = kmeans.labels_
centroids = kmeans.cluster_centers_

# Visualize clusters
colors = ['skyblue', 'pink', 'lightpink', 'wheat', 'lavender', 'hotpink'] # Add more colors if needed
for i in range(k):
    plt.scatter(X[labels == i, 0], X[labels == i, 1], s=70, c=colors[i], label=f'Cluster {i+1}')
plt.scatter(centroids[:, 0], centroids[:,1], s=200, marker='o', c='black', label='Centroids')
plt.xlabel('battery power') #Total energy a battery can store in one time measured in mAh
plt.ylabel('Pixel Resolution Height') 
plt.show()

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans

# Load data into a DataFrame
df = kmeandata

# Extract features to cluster on
X = df[['battery_power', 'pc']].values

# Choose number of clusters
k = 6

# Run K-means clustering
kmeans = KMeans(n_clusters=k, random_state=42).fit(X)

# Get cluster assignments and centroids
labels = kmeans.labels_
centroids = kmeans.cluster_centers_

# Visualize clusters
colors = ['skyblue', 'pink', 'lightpink', 'wheat', 'lavender', 'hotpink'] # Add more colors if needed
for i in range(k):
    plt.scatter(X[labels == i, 0], X[labels == i, 1], s=70, c=colors[i], label=f'Cluster {i+1}')
plt.scatter(centroids[:, 0], centroids[:,1], s=200, marker='o', c='black', label='Centroids')
plt.xlabel('battery power') #Total energy a battery can store in one time measured in mAh
plt.ylabel('Pixel Resolution Height') 
plt.show()

การจัดกลุ่มโทรศัพท์มือถือที่คล้ายกันเข้าด้วยกันตามคุณสมบัติ

In [ ]:
import pandas as pd
import numpy as np
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler
from sklearn.manifold import TSNE
import matplotlib.pyplot as plt
import seaborn as sns

# Load the data
train_data = pd.read_csv('/content/drive/MyDrive/bsc_DPDM_data/Project/train.csv')
# Preprocess the data
X = train_data.drop(['price_range'], axis=1)
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Perform k-means clustering
kmeans = KMeans(n_clusters=4, random_state=42)
kmeans.fit(X_scaled)

# Print the cluster labels
print(kmeans.labels_)

# Apply t-SNE for visualization
tsne = TSNE(n_components=2, perplexity=30, random_state=42)
X_tsne = tsne.fit_transform(X_scaled)

# Create scatter plot of t-SNE visualization with cluster labels
plt.figure(figsize=(10, 8))
sns.scatterplot(x=X_tsne[:,0], y=X_tsne[:,1], hue=kmeans.labels_, palette='viridis')
plt.title('K-means clustering with t-SNE visualization')
plt.show()

# **Association rule**
Association rule มักจะใช้หารูปแบบในผลิตภัณฑ์ที่ลูกค้ามักจะซื้อร่วมกัน อาจใช้ไม่ได้โดยตรงกับปัญหาการจัดประเภทราคามือถือ เราเลยนำ Association rule มาสำรวจความสัมพันธ์ระหว่างคุณลักษณะต่างๆ แทน 

In [ ]:
from mlxtend.frequent_patterns import apriori
from mlxtend.frequent_patterns import association_rules

# Load the data
train_data = pd.read_csv('/content/drive/MyDrive/bsc_DPDM_data/Project/train.csv')

# Preprocess the data
X = train_data.drop('price_range', axis=1)
X = pd.get_dummies(X, columns=['battery_power', 'blue', 'clock_speed', 'dual_sim', 'fc', 'four_g', 'int_memory', 'm_dep', 'mobile_wt', 'n_cores', 'pc', 'px_height', 'px_width', 'ram', 'sc_h', 'sc_w', 'talk_time', 'three_g', 'touch_screen', 'wifi'], drop_first=True)

# Convert the data to a binary matrix
X = (X > 0).astype(int)

# Use Apriori to find frequent itemsets
frequent_itemsets = apriori(X, min_support=0.1, use_colnames=True)

# Use Association Rules to find interesting associations
rules = association_rules(frequent_itemsets, metric="lift", min_threshold=1)

# Print the top 10 rules sorted by lift
print(rules.sort_values('lift', ascending=False).head(10))

In [ ]:
rules.sort_values('lift', ascending=False).head(10)

ตัวอย่าง ถ้าในโทรศัพท์มือมีฟีเจอร์ รองรับ dual_sim, มี 3G และ มีบลูทูธ 
จะมีฟีเจอร์ 4G ด้วย

#PPT presentation

* https://www.canva.com/design/DAFeM3RNkWQ/-mOY5_GGzoae5fT1Hce3hw/view?utm_content=DAFeM3RNkWQ&utm_campaign=designshare&utm_medium=link&utm_source=publishsharelink